In [3]:
import pandas as pd

In [6]:
test = {'num1': [,2,3], 'cat1': ['a','b','a'], 'num2': [4,5,6], 'cat2': ['d','e','f']}

SyntaxError: invalid syntax (<ipython-input-6-13b074df8e79>, line 1)

In [7]:
test

{'num1': [1, 2, 3],
 'cat1': ['a', 'b', 'a'],
 'num2': [4, 5, 6],
 'cat2': ['d', 'e', 'f']}

test.to_array()

In [10]:
pd.DataFrame(test)

,num1,cat1,num2,cat2
0,1,a,4,d
1,2,b,5,e
2,3,a,6,f


In [19]:
a = pd.read_csv("missing_test.csv")

In [20]:
a

,num1,cat1,num2,cat2
0,1.0,a,4,d
1,2.0,b,5,NaN
2,NaN,c,6,g


In [21]:
a.to_numpy()

array([[1.0, 'a', 4, 'd'],
       [2.0, 'b', 5, nan],
       [nan, 'c', 6, 'g']], dtype=object)

In [89]:
test_data = {'num1': [1,2,None], 'cat1': ['a',np.NaN ,'c'], 'num2': [4,None,6], 'cat2': ['d','d','f']}
dataset = pd.DataFrame(data=test_data)

row_length = len(dataset)
column_length = len(dataset.select_dtypes(np.number).columns)

In [27]:
import numpy as np


In [29]:
column_length

2

In [90]:
b = dataset.select_dtypes(exclude=np.number)

In [91]:
b

,cat1,cat2
0,a,d
1,NaN,d
2,c,f


In [34]:
b.items()

<generator object DataFrame.items at 0x7f4136dd9580>

In [41]:
for key, val in b.items():
    print(key)
    print(val)

cat1
0    a
1    b
2    c
Name: cat1, dtype: object
cat2
0    d
1    e
2    f
Name: cat2, dtype: object


In [43]:
for val in b.values:
#     print(key)
    print(val)

['a' 'd']
['b' 'e']
['c' 'f']


In [46]:
for key in b.keys():
    print(key)

cat1
cat2


In [49]:
for key in b.columns:
    print(key)

cat1
cat2


In [57]:
for i in b.itertuples():
    print(i)


Pandas(Index=0, cat1='a', cat2='d')
Pandas(Index=1, cat1='b', cat2='d')
Pandas(Index=2, cat1='c', cat2='f')


In [62]:
for i in b:
    print(set(b[i]))

{'c', 'b', 'a'}
{'f', 'd'}


In [63]:
b

,cat1,cat2
0,a,d
1,b,d
2,c,f


In [73]:
b.isnull()

,cat1,cat2
0,False,False
1,False,False
2,False,False


In [65]:
b.isnull().sum()

cat1    0
cat2    0
dtype: int64

In [167]:
b.isnull().values.any()

cat1     True
cat2    False
dtype: bool

In [75]:
b

,cat1,cat2
0,a,d
1,NaN,d
2,c,f


In [76]:
b.isnull()

,cat1,cat2
0,False,False
1,False,False
2,False,False


In [82]:
b.isna()

,cat1,cat2
0,False,False
1,False,False
2,False,False


In [92]:
dataset

,num1,cat1,num2,cat2
0,1.0,a,4.0,d
1,2.0,NaN,NaN,d
2,NaN,c,6.0,f


In [95]:
dataset.isnull().values.any()

True

In [94]:
dataset.isna()

,num1,cat1,num2,cat2
0,False,False,False,False
1,False,True,True,False
2,True,False,False,False


In [96]:
dataset

,num1,cat1,num2,cat2
0,1.0,a,4.0,d
1,2.0,NaN,NaN,d
2,NaN,c,6.0,f


In [97]:
from sklearn.preprocessing import OneHotEncoder


In [98]:
one = OneHotEncoder()
one.fit_transform(dataset)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [101]:
from utils import Preprocessor

ModuleNotFoundError: No module named 'utils'

In [75]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.base import TransformerMixin
from sklearn.pipeline import make_pipeline


class Preprocessor(TransformerMixin):
    def __init__(self, scaler):
        self.scaler = scaler
        self.numeric_features = None
        self.categorical_features = None
        self.pipeline = None

    def _set_feature_types(self, X):
        self.numeric_features = X.select_dtypes(np.number).columns
        self.categorical_features = X.select_dtypes(exclude=np.number).columns
        return

    # preprocess numeric and categorical features
    def _build_pipeline(self):
        assert (
            self.numeric_features is not None
            and self.categorical_features is not None
        )

        self.pipeline = ColumnTransformer(
            transformers=[
                (
                    "numeric",
                    make_pipeline(
                        SimpleImputer(strategy="median"), self.scaler(),
                    ),
                    self.numeric_features,
                ),
                (
                    "categorical",
                    make_pipeline(
                        SimpleImputer(strategy="most_frequent"),
                        OneHotEncoder(),
                    ),
                    self.categorical_features,
                ),
            ]
        )
        return

    # get column names for dataframe
    def get_column_names(self):
        cats_names = self.pipeline.named_transformers_["categorical"][
            1
        ].get_feature_names(self.categorical_features)
        feat_names = np.concatenate([self.numeric_features, cats_names])

        return feat_names

    def fit(self, X, y=None):
        self._set_feature_types(X)
        self._build_pipeline()
        self.pipeline.fit(X)

        return self

    def transform(self, X):
        transformed_data = self.pipeline.transform(X)
        column_names = self.get_column_names()

        return pd.DataFrame(
            transformed_data.toarray(), columns=column_names, index=X.index,
        )


In [76]:
pre = Preprocessor(scaler=StandardScaler)

In [21]:
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
)


In [108]:
hello = pre.fit_transform(dataset)

In [109]:
hello

,num1,num2,cat1_a,cat1_c,cat2_d,cat2_f
0,-1.224745,-1.224745,1.0,0.0,1.0,0.0
1,1.224745,0.000000,1.0,0.0,1.0,0.0
2,0.000000,1.224745,0.0,1.0,0.0,1.0


In [117]:
hello.bool()

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [116]:
hello.describe()

,num1,num2,cat1_a,cat1_c,cat2_d,cat2_f
count,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
mean,0.000000,0.000000,0.666667,0.333333,0.666667,0.333333
std,1.224745,1.224745,0.577350,0.577350,0.577350,0.577350
min,-1.224745,-1.224745,0.000000,0.000000,0.000000,0.000000
25%,-0.612372,-0.612372,0.500000,0.000000,0.500000,0.000000
50%,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
75%,0.612372,0.612372,1.000000,0.500000,1.000000,0.500000
max,1.224745,1.224745,1.000000,1.000000,1.000000,1.000000


In [118]:
hello.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   num1    3 non-null      float64
 1   num2    3 non-null      float64
 2   cat1_a  3 non-null      float64
 3   cat1_c  3 non-null      float64
 4   cat2_d  3 non-null      float64
 5   cat2_f  3 non-null      float64
dtypes: float64(6)
memory usage: 272.0 bytes


In [120]:
hello.bool

<bound method NDFrame.bool of        num1      num2  cat1_a  cat1_c  cat2_d  cat2_f
0 -1.224745 -1.224745     1.0     0.0     1.0     0.0
1  1.224745  0.000000     1.0     0.0     1.0     0.0
2  0.000000  1.224745     0.0     1.0     0.0     1.0>

In [121]:
hello.dtypes

num1      float64
num2      float64
cat1_a    float64
cat1_c    float64
cat2_d    float64
cat2_f    float64
dtype: object

In [122]:
hello.cat1_a.dtypes

dtype('float64')

In [123]:
hello.isin([0,1]).all()

num1      False
num2      False
cat1_a     True
cat1_c     True
cat2_d     True
cat2_f     True
dtype: bool

In [124]:
num = dataset.select_dtypes(np.number)

In [125]:
num

,num1,num2
0,1.0,4.0
1,2.0,NaN
2,NaN,6.0


In [127]:
hello.drop(num)

KeyError: "['num1' 'num2'] not found in axis"

In [128]:
num.columns

Index(['num1', 'num2'], dtype='object')

In [129]:
num

,num1,num2
0,1.0,4.0
1,2.0,NaN
2,NaN,6.0


In [132]:
yay = hello.drop(num, axis=1)

In [131]:
hello

,num1,num2,cat1_a,cat1_c,cat2_d,cat2_f
0,-1.224745,-1.224745,1.0,0.0,1.0,0.0
1,1.224745,0.000000,1.0,0.0,1.0,0.0
2,0.000000,1.224745,0.0,1.0,0.0,1.0


In [133]:
yay

,cat1_a,cat1_c,cat2_d,cat2_f
0,1.0,0.0,1.0,0.0
1,1.0,0.0,1.0,0.0
2,0.0,1.0,0.0,1.0


In [142]:
yay.isin([0,1]).values

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [157]:
yay.isin([0,1]).all().all()

True

In [147]:
type(yay.isin([0,1]))

pandas.core.frame.DataFrame

In [148]:
type(yay.isin([0,1]).values)

numpy.ndarray

In [149]:
type(yay.isin([0,1]).all())

pandas.core.series.Series

In [150]:
type(yay.isin([0,1]).all().any())

numpy.bool_

In [156]:
hello.isin([0,1]).all().all()

False

In [158]:
hello

,num1,num2,cat1_a,cat1_c,cat2_d,cat2_f
0,-1.224745,-1.224745,1.0,0.0,1.0,0.0
1,1.224745,0.000000,1.0,0.0,1.0,0.0
2,0.000000,1.224745,0.0,1.0,0.0,1.0


In [159]:
hello.columns

Index(['num1', 'num2', 'cat1_a', 'cat1_c', 'cat2_d', 'cat2_f'], dtype='object')

In [166]:
hello.columns.isin(['num1', 'num2']).all()

False

In [168]:
dataset

,num1,cat1,num2,cat2
0,1.0,a,4.0,d
1,2.0,NaN,NaN,d
2,NaN,c,6.0,f


In [9]:
from sklearn.feature_selection import SelectKBest, f_classif


In [10]:
s = SelectKBest(f_classif, k=4)

In [171]:
s.fit(dataset.drop)

TypeError: fit() missing 1 required positional argument: 'y'

In [3]:
mark = pd.read_csv("marketing_train.csv")

In [2]:
import pandas as pd

In [4]:
mark

,age,job,marital_status,education,default,balance,housing_loan,personal_loan,contact_channel,last_contact_day,last_contact_month,contact_duration,campaign,gap_between_campaigns,previous,outcome_previous_campaign,insurance_subscribe
0,41,blue-collar,married,primary,no,849,yes,no,unknown,15,may,72,1,-1,0,unknown,0
1,49,technician,married,primary,no,1415,yes,no,cellular,30,jul,269,2,-1,0,unknown,0
2,42,admin.,married,secondary,no,3842,no,no,cellular,31,jul,130,4,-1,0,unknown,0
3,37,management,single,tertiary,no,-119,yes,no,unknown,11,jun,375,11,-1,0,unknown,0
4,56,blue-collar,married,primary,no,3498,no,no,cellular,15,apr,264,2,-1,0,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36163,44,housemaid,single,primary,no,1059,no,no,unknown,18,jun,2093,1,-1,0,unknown,1
36164,23,student,single,tertiary,no,508,no,no,cellular,8,sep,210,1,92,1,failure,0
36165,34,technician,divorced,tertiary,no,1317,yes,no,cellular,15,may,239,1,-1,0,unknown,0
36166,33,retired,married,secondary,no,165,no,no,unknown,7,may,111,1,-1,0,unknown,0


In [5]:
drop_ins = mark.drop("insurance_subscribe", axis=1)

In [6]:
drop_ins

,age,job,marital_status,education,default,balance,housing_loan,personal_loan,contact_channel,last_contact_day,last_contact_month,contact_duration,campaign,gap_between_campaigns,previous,outcome_previous_campaign
0,41,blue-collar,married,primary,no,849,yes,no,unknown,15,may,72,1,-1,0,unknown
1,49,technician,married,primary,no,1415,yes,no,cellular,30,jul,269,2,-1,0,unknown
2,42,admin.,married,secondary,no,3842,no,no,cellular,31,jul,130,4,-1,0,unknown
3,37,management,single,tertiary,no,-119,yes,no,unknown,11,jun,375,11,-1,0,unknown
4,56,blue-collar,married,primary,no,3498,no,no,cellular,15,apr,264,2,-1,0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36163,44,housemaid,single,primary,no,1059,no,no,unknown,18,jun,2093,1,-1,0,unknown
36164,23,student,single,tertiary,no,508,no,no,cellular,8,sep,210,1,92,1,failure
36165,34,technician,divorced,tertiary,no,1317,yes,no,cellular,15,may,239,1,-1,0,unknown
36166,33,retired,married,secondary,no,165,no,no,unknown,7,may,111,1,-1,0,unknown


In [14]:
s.fit(mark.select_dtypes(exclude=np.number), mark["insurance_subscribe"])

ValueError: could not convert string to float: 'blue-collar'

In [13]:
import numpy as np

In [18]:
mark.select_dtypes(exclude=np.number)

,job,marital_status,education,default,housing_loan,personal_loan,contact_channel,last_contact_month,outcome_previous_campaign
0,blue-collar,married,primary,no,yes,no,unknown,may,unknown
1,technician,married,primary,no,yes,no,cellular,jul,unknown
2,admin.,married,secondary,no,no,no,cellular,jul,unknown
3,management,single,tertiary,no,yes,no,unknown,jun,unknown
4,blue-collar,married,primary,no,no,no,cellular,apr,unknown
...,...,...,...,...,...,...,...,...,...
36163,housemaid,single,primary,no,no,no,unknown,jun,unknown
36164,student,single,tertiary,no,no,no,cellular,sep,failure
36165,technician,divorced,tertiary,no,yes,no,cellular,may,unknown
36166,retired,married,secondary,no,no,no,unknown,may,unknown


In [23]:
from sklearn.feature_selection import SelectFromModel

In [24]:
pre_fit = pre.fit(mark.select_dtypes(exclude=np.number), mark["insurance_subscribe"])

In [25]:
pre_fit

In [26]:
select_model = SelectFromModel(pre_fit, prefit=True)

In [27]:
select_model

SelectFromModel(estimator=<__main__.Preprocessor object at 0x7f52dfc708e0>,
                prefit=True)

In [28]:
select_model.transform(mark.select_dtypes(exclude=np.number))

AttributeError: 'Preprocessor' object has no attribute '_get_tags'

In [29]:
mark

,age,job,marital_status,education,default,balance,housing_loan,personal_loan,contact_channel,last_contact_day,last_contact_month,contact_duration,campaign,gap_between_campaigns,previous,outcome_previous_campaign,insurance_subscribe
0,41,blue-collar,married,primary,no,849,yes,no,unknown,15,may,72,1,-1,0,unknown,0
1,49,technician,married,primary,no,1415,yes,no,cellular,30,jul,269,2,-1,0,unknown,0
2,42,admin.,married,secondary,no,3842,no,no,cellular,31,jul,130,4,-1,0,unknown,0
3,37,management,single,tertiary,no,-119,yes,no,unknown,11,jun,375,11,-1,0,unknown,0
4,56,blue-collar,married,primary,no,3498,no,no,cellular,15,apr,264,2,-1,0,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36163,44,housemaid,single,primary,no,1059,no,no,unknown,18,jun,2093,1,-1,0,unknown,1
36164,23,student,single,tertiary,no,508,no,no,cellular,8,sep,210,1,92,1,failure,0
36165,34,technician,divorced,tertiary,no,1317,yes,no,cellular,15,may,239,1,-1,0,unknown,0
36166,33,retired,married,secondary,no,165,no,no,unknown,7,may,111,1,-1,0,unknown,0


In [30]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

In [31]:
X, y = load_iris(return_X_y=True)

In [32]:
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [33]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [34]:
X.shape

(150, 4)

In [35]:
mark

,age,job,marital_status,education,default,balance,housing_loan,personal_loan,contact_channel,last_contact_day,last_contact_month,contact_duration,campaign,gap_between_campaigns,previous,outcome_previous_campaign,insurance_subscribe
0,41,blue-collar,married,primary,no,849,yes,no,unknown,15,may,72,1,-1,0,unknown,0
1,49,technician,married,primary,no,1415,yes,no,cellular,30,jul,269,2,-1,0,unknown,0
2,42,admin.,married,secondary,no,3842,no,no,cellular,31,jul,130,4,-1,0,unknown,0
3,37,management,single,tertiary,no,-119,yes,no,unknown,11,jun,375,11,-1,0,unknown,0
4,56,blue-collar,married,primary,no,3498,no,no,cellular,15,apr,264,2,-1,0,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36163,44,housemaid,single,primary,no,1059,no,no,unknown,18,jun,2093,1,-1,0,unknown,1
36164,23,student,single,tertiary,no,508,no,no,cellular,8,sep,210,1,92,1,failure,0
36165,34,technician,divorced,tertiary,no,1317,yes,no,cellular,15,may,239,1,-1,0,unknown,0
36166,33,retired,married,secondary,no,165,no,no,unknown,7,may,111,1,-1,0,unknown,0


In [36]:
from sklearn.model_selection import train_test_split

In [119]:
def split_test(dataset, target):
    X = dataset.drop(target, axis=1)
    y = dataset[target]

    X_train, X_validation, y_train, y_validation = train_test_split(X, y)

    return X_train, X_validation, y_train, y_validation

In [38]:
(X_train, X_validation, y_train, y_validation) = split_test(mark, "insurance_subscribe")

In [39]:
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X_train, y_train)
model = SelectFromModel(lsvc, prefit=True)

ValueError: could not convert string to float: 'unknown'

In [40]:
X_train

,age,job,marital_status,education,default,balance,housing_loan,personal_loan,contact_channel,last_contact_day,last_contact_month,contact_duration,campaign,gap_between_campaigns,previous,outcome_previous_campaign
24086,59,retired,married,primary,no,-266,yes,yes,cellular,31,jul,266,11,-1,0,unknown
257,48,technician,married,secondary,no,127,no,yes,cellular,7,aug,524,5,-1,0,unknown
11442,39,management,married,secondary,no,18,yes,no,cellular,21,aug,102,1,91,1,failure
4862,34,blue-collar,single,secondary,no,13683,yes,no,unknown,3,jun,115,3,-1,0,unknown
13075,46,blue-collar,single,primary,no,2851,yes,no,unknown,19,may,122,2,-1,0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5325,51,management,married,tertiary,no,3604,no,yes,telephone,8,jul,287,1,-1,0,unknown
36027,72,self-employed,married,tertiary,no,132,no,no,cellular,27,apr,444,1,-1,0,unknown
26830,31,technician,married,tertiary,no,19,yes,no,cellular,28,jul,243,31,-1,0,unknown
28771,29,student,single,tertiary,no,246,no,no,cellular,3,feb,269,1,-1,0,unknown


In [41]:
y_train

24086    0
257      0
11442    0
4862     0
13075    0
        ..
5325     0
36027    1
26830    0
28771    0
27315    0
Name: insurance_subscribe, Length: 27126, dtype: int64

In [42]:
pre

In [67]:
x_tr = pre.fit_transform(X_train)

In [46]:
x_tr.transform(X_train)

,age,balance,last_contact_day,contact_duration,campaign,gap_between_campaigns,previous,job_admin.,job_blue-collar,job_entrepreneur,...,last_contact_month_jun,last_contact_month_mar,last_contact_month_may,last_contact_month_nov,last_contact_month_oct,last_contact_month_sep,outcome_previous_campaign_failure,outcome_previous_campaign_other,outcome_previous_campaign_success,outcome_previous_campaign_unknown
24086,1.710954,-0.530540,1.822529,0.042483,2.626750,-0.410213,-0.298407,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
257,0.668569,-0.402027,-1.062616,1.053642,0.712293,-0.410213,-0.298407,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11442,-0.184291,-0.437671,0.620385,-0.600269,-0.564012,0.509409,0.228482,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4862,-0.658103,4.030845,-1.543473,-0.549319,0.074141,-0.410213,-0.298407,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
13075,0.479045,0.488733,0.379956,-0.521884,-0.244936,-0.410213,-0.298407,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5325,0.952856,0.734967,-0.942402,0.124787,-0.564012,-0.410213,-0.298407,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
36027,2.942864,-0.400392,1.341671,0.740104,-0.564012,-0.410213,-0.298407,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
26830,-0.942390,-0.437344,1.461886,-0.047659,9.008275,-0.410213,-0.298407,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
28771,-1.131914,-0.363113,-1.543473,0.054241,-0.564012,-0.410213,-0.298407,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [53]:
model = SelectFromModel(x_tr)

In [54]:
model

SelectFromModel(estimator=<__main__.Preprocessor object at 0x7f52dfc708e0>)

In [55]:
new = model.transform(x_tr)

AttributeError: 'Preprocessor' object has no attribute '_get_tags'

In [57]:
sfm = SelectFromModel(LinearSVC(penalty="l1"))

In [59]:
sfm.fit(x_tr, y_train)

TypeError: float() argument must be a string or a number, not 'Preprocessor'

In [62]:
from sklearn.feature_selection import VarianceThreshold

In [63]:
mark

,age,job,marital_status,education,default,balance,housing_loan,personal_loan,contact_channel,last_contact_day,last_contact_month,contact_duration,campaign,gap_between_campaigns,previous,outcome_previous_campaign,insurance_subscribe
0,41,blue-collar,married,primary,no,849,yes,no,unknown,15,may,72,1,-1,0,unknown,0
1,49,technician,married,primary,no,1415,yes,no,cellular,30,jul,269,2,-1,0,unknown,0
2,42,admin.,married,secondary,no,3842,no,no,cellular,31,jul,130,4,-1,0,unknown,0
3,37,management,single,tertiary,no,-119,yes,no,unknown,11,jun,375,11,-1,0,unknown,0
4,56,blue-collar,married,primary,no,3498,no,no,cellular,15,apr,264,2,-1,0,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36163,44,housemaid,single,primary,no,1059,no,no,unknown,18,jun,2093,1,-1,0,unknown,1
36164,23,student,single,tertiary,no,508,no,no,cellular,8,sep,210,1,92,1,failure,0
36165,34,technician,divorced,tertiary,no,1317,yes,no,cellular,15,may,239,1,-1,0,unknown,0
36166,33,retired,married,secondary,no,165,no,no,unknown,7,may,111,1,-1,0,unknown,0


In [64]:
selector = VarianceThreshold()

In [69]:
a = selector.fit_transform(x_tr)

In [70]:
a.shape

(27126, 51)

In [71]:
x_tr.shape

(27126, 51)

In [72]:
tit = pd.read_csv("source_kaggle_titanic_train.csv")

In [73]:
tit

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [78]:
t1 = pre.fit_transform(tit)

In [79]:
t1

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,"Name_Abbing, Mr. Anthony","Name_Abbott, Mr. Rossmore Edward","Name_Abbott, Mrs. Stanton (Rosa Hunt)",...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
0,-1.730108,-0.789272,0.827377,-0.565736,0.432793,-0.473674,-0.502445,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-1.726220,1.266990,-1.566107,0.663861,0.432793,-0.473674,0.786845,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-1.722332,1.266990,0.827377,-0.258337,-0.474545,-0.473674,-0.488854,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-1.718444,1.266990,-1.566107,0.433312,0.432793,-0.473674,0.420730,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-1.714556,-0.789272,0.827377,0.433312,-0.474545,-0.473674,-0.486337,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,1.714556,-0.789272,-0.369365,-0.181487,-0.474545,-0.473674,-0.386671,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
887,1.718444,1.266990,-1.566107,-0.796286,-0.474545,-0.473674,-0.044381,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
888,1.722332,-0.789272,0.827377,-0.104637,0.432793,2.008933,-0.176263,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
889,1.726220,1.266990,-1.566107,-0.258337,-0.474545,-0.473674,-0.044381,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [80]:
t2 = selector.fit_transform(t1)

In [81]:
t2

array([[-1.73010796, -0.78927234,  0.82737724, ...,  0.        ,
         0.        ,  1.        ],
       [-1.72622007,  1.2669898 , -1.56610693, ...,  1.        ,
         0.        ,  0.        ],
       [-1.72233219,  1.2669898 ,  0.82737724, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.72233219, -0.78927234,  0.82737724, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.72622007,  1.2669898 , -1.56610693, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.73010796, -0.78927234,  0.82737724, ...,  0.        ,
         1.        ,  0.        ]])

In [82]:
t2.shape

(891, 1731)

In [83]:
tit

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [87]:
length = len(tit.Name.unique())

In [88]:
tit.unique()

AttributeError: 'DataFrame' object has no attribute 'unique'

In [91]:
tit

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [103]:
pd.unique(tit['Name'])

array(['Braund, Mr. Owen Harris',
       'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
       'Heikkinen, Miss. Laina',
       'Futrelle, Mrs. Jacques Heath (Lily May Peel)',
       'Allen, Mr. William Henry', 'Moran, Mr. James',
       'McCarthy, Mr. Timothy J', 'Palsson, Master. Gosta Leonard',
       'Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)',
       'Nasser, Mrs. Nicholas (Adele Achem)',
       'Sandstrom, Miss. Marguerite Rut', 'Bonnell, Miss. Elizabeth',
       'Saundercock, Mr. William Henry', 'Andersson, Mr. Anders Johan',
       'Vestrom, Miss. Hulda Amanda Adolfina',
       'Hewlett, Mrs. (Mary D Kingcome) ', 'Rice, Master. Eugene',
       'Williams, Mr. Charles Eugene',
       'Vander Planke, Mrs. Julius (Emelia Maria Vandemoortele)',
       'Masselmani, Mrs. Fatima', 'Fynney, Mr. Joseph J',
       'Beesley, Mr. Lawrence', 'McGowan, Miss. Anna "Annie"',
       'Sloper, Mr. William Thompson', 'Palsson, Miss. Torborg Danira',
       'Asplund, Mrs. Carl Oscar 

In [114]:
np.unique(tit[["Name"]], axis=0)

TypeError: The axis argument to unique is not supported for dtype object

In [115]:
t1

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,"Name_Abbing, Mr. Anthony","Name_Abbott, Mr. Rossmore Edward","Name_Abbott, Mrs. Stanton (Rosa Hunt)",...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
0,-1.730108,-0.789272,0.827377,-0.565736,0.432793,-0.473674,-0.502445,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-1.726220,1.266990,-1.566107,0.663861,0.432793,-0.473674,0.786845,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-1.722332,1.266990,0.827377,-0.258337,-0.474545,-0.473674,-0.488854,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-1.718444,1.266990,-1.566107,0.433312,0.432793,-0.473674,0.420730,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-1.714556,-0.789272,0.827377,0.433312,-0.474545,-0.473674,-0.486337,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,1.714556,-0.789272,-0.369365,-0.181487,-0.474545,-0.473674,-0.386671,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
887,1.718444,1.266990,-1.566107,-0.796286,-0.474545,-0.473674,-0.044381,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
888,1.722332,-0.789272,0.827377,-0.104637,0.432793,2.008933,-0.176263,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
889,1.726220,1.266990,-1.566107,-0.258337,-0.474545,-0.473674,-0.044381,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [116]:
from sklearn.feature_selection import SelectKBest, chi2

In [117]:
mod1 = SelectKBest(score_func=chi2, k=4)

In [121]:
mod1.fit_transform(X_train, y_train)

ValueError: Input X must be non-negative.

In [120]:
(X_train, X_validation, y_train, y_validation) = split_test(t1, "Survived")

In [122]:
X_train

,PassengerId,Pclass,Age,SibSp,Parch,Fare,"Name_Abbing, Mr. Anthony","Name_Abbott, Mr. Rossmore Edward","Name_Abbott, Mrs. Stanton (Rosa Hunt)","Name_Abelson, Mr. Samuel",...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
289,-0.606510,0.827377,-0.565736,-0.474545,-0.473674,-0.492378,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
24,-1.636799,0.827377,-1.641634,2.247470,0.767630,-0.224083,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
820,1.457956,-1.566107,1.739759,0.432793,0.767630,1.234172,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
159,-1.111935,0.827377,-0.104637,6.784163,2.008933,0.751946,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
156,-1.123598,0.827377,-1.026836,-0.474545,-0.473674,-0.492714,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,0.758137,0.827377,-0.719436,-0.474545,-0.473674,-0.490280,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
813,1.430741,0.827377,-1.795334,3.154809,2.008933,-0.018709,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
860,1.613471,0.827377,0.894411,1.340132,-0.473674,-0.364355,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
421,-0.093309,0.827377,-0.642586,-0.474545,-0.473674,-0.492714,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [123]:
tit

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [125]:
pre.fit_transform(tit)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,"Name_Abbing, Mr. Anthony","Name_Abbott, Mr. Rossmore Edward","Name_Abbott, Mrs. Stanton (Rosa Hunt)",...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
0,-1.730108,-0.789272,0.827377,-0.565736,0.432793,-0.473674,-0.502445,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-1.726220,1.266990,-1.566107,0.663861,0.432793,-0.473674,0.786845,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-1.722332,1.266990,0.827377,-0.258337,-0.474545,-0.473674,-0.488854,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-1.718444,1.266990,-1.566107,0.433312,0.432793,-0.473674,0.420730,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-1.714556,-0.789272,0.827377,0.433312,-0.474545,-0.473674,-0.486337,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,1.714556,-0.789272,-0.369365,-0.181487,-0.474545,-0.473674,-0.386671,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
887,1.718444,1.266990,-1.566107,-0.796286,-0.474545,-0.473674,-0.044381,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
888,1.722332,-0.789272,0.827377,-0.104637,0.432793,2.008933,-0.176263,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
889,1.726220,1.266990,-1.566107,-0.258337,-0.474545,-0.473674,-0.044381,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [126]:
tit

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [127]:
from sklearn.decomposition import PCA

In [128]:
pca = PCA(n_components=3)

In [130]:
fit = pca.fit(X_train)

In [131]:
print(fit.explained_variance_ratio_, fit.components_)

[0.20882428 0.16541906 0.11026857] [[ 0.07807033 -0.63616781  0.4145633  ...  0.08785196 -0.03299648
  -0.05485547]
 [-0.11558845  0.01590956 -0.38502845 ...  0.00790207 -0.00174367
  -0.0061584 ]
 [ 0.98184384  0.06405306 -0.02130775 ... -0.00613292 -0.01261506
   0.01874798]]


In [133]:
from sklearn.ensemble import ExtraTreesClassifier

In [134]:
mod2 = ExtraTreesClassifier(n_estimators=100)

In [136]:
mod2.fit(X_train, y_train)

ValueError: Unknown label type: 'continuous'

In [139]:
tit["Name"].is_unique

True

In [140]:
tit["Survived"].is_unique

False

In [142]:
len(tit["Name"].unique())

891

In [145]:
len(tit["Survived"].unique())

2

In [153]:
remove_cols = []
for col in tit.columns:
    if (tit[col].dtype == object) and (len(tit[col].unique()) > 30):
        remove_cols.append(col)

In [154]:
remove_cols

['Name', 'Ticket', 'Cabin']

In [151]:
tit['Survived'].dtype  == object

False

In [165]:
remove_cols = []
for col in tit:
    if (tit[col].dtype == object) and (len(tit[col].unique()) > 30):
        remove_cols.append(col)

In [166]:
remove_cols

['Name', 'Ticket', 'Cabin']

In [167]:
len(tit)

891

In [168]:
tit

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [169]:
from sklearn.pipeline import Pipeline

In [173]:
estimators = [('feature_selector', FeatureSelector()), ('pre', Preprocessor(scaler=StandardScaler))]

In [174]:
class FeatureSelector(TransformerMixin):
    def __init__(self):
        self.dataset = None

    def _remove_features(self, X):
        remove_columns = []
        for column in X:
            if len(X[column].unique()) > len(X)/2:
                remove_columns.append(column)

        return remove_columns

    def fit(self, X):
        self.dataset = X.drop(columns=self._remove_features(X))

        return self

    def transform(self, X):
        #### 여기서 애초에 fit하면 dataframe으로 나오고.. 굳이 fit, transform이라는 함수를 만들어야 하나?
        return pd.DataFrame(self.dataset)


In [175]:
pipe = Pipeline(estimators)

In [176]:
pipe

Pipeline(steps=[('feature_selector',
                 <__main__.FeatureSelector object at 0x7f52d829cdc0>),
                ('pre', <__main__.Preprocessor object at 0x7f52d829c970>)])

In [177]:
pipe.fit(tit)

Pipeline(steps=[('feature_selector',
                 <__main__.FeatureSelector object at 0x7f52d829cdc0>),
                ('pre', <__main__.Preprocessor object at 0x7f52d829c970>)])

In [178]:
pipe.transform(tit)

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Cabin_A10,Cabin_A14,...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
0,-0.789272,0.827377,-0.565736,0.432793,-0.473674,-0.502445,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.266990,-1.566107,0.663861,0.432793,-0.473674,0.786845,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.266990,0.827377,-0.258337,-0.474545,-0.473674,-0.488854,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.266990,-1.566107,0.433312,0.432793,-0.473674,0.420730,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.789272,0.827377,0.433312,-0.474545,-0.473674,-0.486337,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,-0.789272,-0.369365,-0.181487,-0.474545,-0.473674,-0.386671,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
887,1.266990,-1.566107,-0.796286,-0.474545,-0.473674,-0.044381,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
888,-0.789272,0.827377,-0.104637,0.432793,2.008933,-0.176263,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
889,1.266990,-1.566107,-0.258337,-0.474545,-0.473674,-0.044381,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [179]:
tit

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [180]:
tit.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [181]:
tit.columns == tit.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [182]:
import unittest

In [185]:
tit.name

AttributeError: 'DataFrame' object has no attribute 'name'

In [194]:
tit.columns.all() == t1.columns.all()

False

In [195]:
tit.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [196]:
t1.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare',
       'Name_Abbing, Mr. Anthony', 'Name_Abbott, Mr. Rossmore Edward',
       'Name_Abbott, Mrs. Stanton (Rosa Hunt)',
       ...
       'Cabin_F G73', 'Cabin_F2', 'Cabin_F33', 'Cabin_F38', 'Cabin_F4',
       'Cabin_G6', 'Cabin_T', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object', length=1731)

In [201]:
tit[tit["Survived"].isnull()].index.tolist()

[]

In [207]:
tit.index.isnull()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,